<a href="https://colab.research.google.com/github/alexyev/pennylane_projects/blob/master/Plugins_and_Hybrid_Computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install pennylane
!pip install pennylane-sf
import pennylane as qml
from pennylane import numpy as np

In [ ]:
dev_fock = qml.device('strawberryfields.fock', wires=2, cutoff_dim=2)

In [ ]:
@qml.qnode(dev_fock, diff_method='parameter-shift')
def photon_redirection(params):
  qml.FockState(1, wires=0)
  qml.Beamsplitter(params[0], params[1], wires=[0, 1])
  return qml.expval(qml.NumberOperator(1))

In [ ]:
#since this is a maximization problem, we will need to set the cost function to minimize the negative of our circuit output

def cost(params):
  return -photon_redirection(params)

In [ ]:
init_params = np.array([0.01, 0.01], requires_grad=True)
print(cost(init_params))

-9.999666671111081e-05


In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
steps = 100
params = init_params

In [ ]:
for i in range(steps):
  params = opt.step(cost, params)

  if (i + 1) % 5 == 0:
    print('Cost after step {: 5d}: {: .7f}'.format(i + 1, cost(params)))

print('Optimized rotation angles: {}'.format(params))

Cost after step     5: -0.0349558
Cost after step    10: -0.9969017
Cost after step    15: -1.0000000
Cost after step    20: -1.0000000
Cost after step    25: -1.0000000
Cost after step    30: -1.0000000
Cost after step    35: -1.0000000
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [1.57079633 0.01      ]


In [17]:
dev_qubit = qml.device('default.qubit', wires=1)
dev_fock = qml.device('strawberryfields.fock', wires=2, cutoff_dim=10)

In [18]:
@qml.qnode(dev_qubit)
def qubit_rotation(phi1, phi2):
  qml.RX(phi1, wires=0)
  qml.RY(phi2, wires=0)
  return qml.expval(qml.PauliZ(0))

In [19]:
@qml.qnode(dev_fock, diff_method='parameter-shift')
def photon_redirection(params):
  qml.FockState(1, wires=0)
  qml.Beamsplitter(params[0], params[1], wires=[0, 1])
  return qml.expval(qml.NumberOperator(1))

In [20]:
def squared_difference(x, y):
  return np.abs(x - y) ** 2

In [21]:
def cost(params, phi1=0.5, phi2=0.1):
  qubit_result = qubit_rotation(phi1, phi2)
  photon_result = photon_redirection(params)
  return squared_difference(qubit_result, photon_result)

In [29]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
steps = 100
params = np.array([0.01, 0.01], requires_grad=True)

for i in range(steps):
  params = opt.step(cost, params)

  if (i + 1) % 5 == 0:
    print('Cost after step {:5d}: {: .7f}'.format(i + 1, cost(params)))

print('Optimized rotation angles: {}'.format(params))


Cost after step     5:  0.2154539
Cost after step    10:  0.0000982
Cost after step    15:  0.0000011
Cost after step    20:  0.0000000
Cost after step    25:  0.0000000
Cost after step    30:  0.0000000
Cost after step    35:  0.0000000
Cost after step    40:  0.0000000
Cost after step    45:  0.0000000
Cost after step    50:  0.0000000
Cost after step    55:  0.0000000
Cost after step    60:  0.0000000
Cost after step    65:  0.0000000
Cost after step    70:  0.0000000
Cost after step    75:  0.0000000
Cost after step    80:  0.0000000
Cost after step    85:  0.0000000
Cost after step    90:  0.0000000
Cost after step    95:  0.0000000
Cost after step   100:  0.0000000
Optimized rotation angles: [1.20671364 0.01      ]


In [30]:
result = [1.20671364, 0.01]
print(photon_redirection(result))
print(qubit_rotation(0.5, 0.1))

0.8731983021146449
0.8731983044562817
